![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_Marian.ipynb)

## Import ONNX Marian models from HuggingFace 🤗 into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `MarianTransformer` is only available since in `Spark NLP 5.2.0` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import Marian models via `MarianMTModel`. These models are usually under `Text2Text Generation` category and have `marian` in their labels
- Reference: [MarianMT](https://huggingface.co/docs/transformers/model_doc/marian)
- Some [example models](https://huggingface.co/models?other=marian)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.35.2`. This doesn't mean it won't work with the future releases
- We will also need `sentencepiece` for tokenization.

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.35.2 optimum sentencepiece onnx==1.14.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 889.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.7/454.7 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [Helsinki-NLP/opus-mt-en-bg](https://huggingface.co/Helsinki-NLP/opus-mt-en-bg) model from HuggingFace as an example
- In addition to `MarianMTModel` we also need to save the tokenizer. This is the same for every model, these are assets needed for tokenization inside Spark NLP.
- If we want to optimize the model, a GPU will be needed. Make sure to select the correct runtime.
0

In [ ]:
import transformers
# Model name, either HF (e.g. "Helsinki-NLP/opus-mt-en-bg") or a local path
MODEL_NAME = "Helsinki-NLP/opus-mt-en-bg"


# Path to store the exported models
EXPORT_PATH = "onnx_models/mt_en_bg_onnx"

In [ ]:
# Export the model to ONNX using optimum

# Export with optimizations (uncomment next line)
# !optimum-cli export onnx --task text2text-generation-with-past --model {MODEL_NAME} --optimize O2 {EXPORT_PATH}
# IMPORTANT - there is a bug in onnxruntime which crashes it when trying to optimize a T5 small model (or any derivative of it)
# There are two ways to addess the problem:
# 1. Go to onnx_model_bert.py in the onnxruntime module (the full path depends on the module version),
#    find the BertOnnxModel class and comment the following line in the constructor:
#    assert (num_heads == 0 and hidden_size == 0) or (num_heads > 0 and hidden_size % num_heads == 0)
# 2. Disable optimization by removing '--optimize O2' (use line below).

# Export without optimizations
!optimum-cli export onnx --task text2text-generation-with-past --model {MODEL_NAME} {EXPORT_PATH}

2023-12-09 15:36:35.997046: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-09 15:36:35.997138: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-09 15:36:35.997190: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-09 15:36:39.395061: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Framework not specified. Using pt to export to ONNX.
/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip instal

Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {EXPORT_PATH}

total 1231176
-rw-r--r-- 1 root root      1384 Dec  9 15:36 config.json
-rw-r--r-- 1 root root 355460094 Dec  9 15:37 decoder_model_merged.onnx
-rw-r--r-- 1 root root 355224613 Dec  9 15:37 decoder_model.onnx
-rw-r--r-- 1 root root 342573852 Dec  9 15:37 decoder_with_past_model.onnx
-rw-r--r-- 1 root root 203194157 Dec  9 15:36 encoder_model.onnx
-rw-r--r-- 1 root root       288 Dec  9 15:36 generation_config.json
-rw-r--r-- 1 root root    791438 Dec  9 15:36 source.spm
-rw-r--r-- 1 root root        74 Dec  9 15:36 special_tokens_map.json
-rw-r--r-- 1 root root    999053 Dec  9 15:36 target.spm
-rw-r--r-- 1 root root       818 Dec  9 15:36 tokenizer_config.json
-rw-r--r-- 1 root root   2451253 Dec  9 15:36 vocab.json


- As you can see, we need to move the sentence piece models `*.spm` from the tokenizer to assets folder which Spark NLP will look for
- We also need to process `vocab.json` for the tokenizer vocabulary. The Spark NLP Annotator expects a `vocab.txt` with one word per line.

In [ ]:
! mkdir -p {EXPORT_PATH}/assets
! mv -t {EXPORT_PATH}/assets {EXPORT_PATH}/*.spm

In [ ]:
import json
output_json = json.load(open(f"{EXPORT_PATH}/vocab.json"))

with open(f"{EXPORT_PATH}/assets/vocab.txt", "w") as f:
    for key in output_json.keys():
        print(key, file=f)

In [ ]:
!ls -l {EXPORT_PATH}/assets

total 2528
-rw-r--r-- 1 root root 791438 Dec  9 15:36 source.spm
-rw-r--r-- 1 root root 999053 Dec  9 15:36 target.spm
-rw-r--r-- 1 root root 792353 Dec  9 15:38 vocab.txt


## Import and Save Marian in Spark NLP

- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.2.0
setup Colab for PySpark 3.2.3 and Spark NLP 5.2.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.5/548.5 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 20.0 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp

# let's start Spark with Spark NLP
spark = sparknlp.start()

- Let's use `loadSavedModel` functon in `MarianTransformer` which allows us to load the ONNX model
- Most params will be set automatically. They can also be set later after loading the model in `MarianTransformer` during runtime, so don't worry about setting them now
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.

In [ ]:
from sparknlp.annotator import *

marian = MarianTransformer.loadSavedModel(EXPORT_PATH, spark)

Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
marian.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your ONNX Marian model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp

total 547408
-rw-r--r-- 1 root root 355514472 Dec  9 15:40 decoder.onxx
-rw-r--r-- 1 root root 203225300 Dec  9 15:39 encoder.onxx
-rw-r--r-- 1 root root    791438 Dec  9 15:40 marian_spp_src
-rw-r--r-- 1 root root    999053 Dec  9 15:40 marian_spp_trg
drwxr-xr-x 2 root root      4096 Dec  9 15:39 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny Marian model 😊

In [ ]:
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline

test_data = spark.createDataFrame([
    (1, "Rome (Italian and Latin: Roma [ˈroːma] ⓘ) is the capital city of Italy. It is also the capital of the Lazio region, the centre of the Metropolitan City of Rome, and a special comune named Comune di Roma Capitale.s listed by UNESCO as a World Heritage Site.[17] The host city for the 1960 Summer Olympics, Rome is also the seat of several specialised agencies of the United Nations, such as the Food and Agriculture Organization (FAO), the World Food Programme (WFP) and the International Fund for Agricultural Development (IFAD). The city also hosts the Secretariat of the Parliamentary Assembly of the Union for the Mediterranean[18] (UfM) as well as the headquarters of many international businesses, such as Eni, Enel, TIM, Leonardo, and banks such as BNL. Numerous companies are based within Rome's EUR business district, such as the luxury fashion house Fendi located in the Palazzo della Civiltà Italiana. The presence of renowned international brands in the city has made Rome an important centre of fashion and design, and the Cinecittà Studios have been the set of many Academy Award–winning movies."),
]).toDF("id", "text")


document_assembler = DocumentAssembler() \
    .setInputCol("text")\
    .setOutputCol("document")

marian = MarianTransformer.load(f"{MODEL_NAME}_spark_nlp") \
    .setInputCols(["document"])\
    .setOutputCol("translation")\
    .setMaxInputLength(512)

pipeline = Pipeline().setStages([document_assembler, marian])

result = pipeline.fit(test_data).transform(test_data)
result.select("translation.result").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                        |
+----------------------------------------------------------------------------------------------------------------------------------------------+
|[В началото на 2004 г., в началото на 2007 г., в началото на 2007 г., се провеждаше и конгрес на тема „Статии за развитие на земеделието“ и „]|
+----------------------------------------------------------------------------------------------------------------------------------------------+



That's it! You can now go wild and use hundreds of Marian models from HuggingFace 🤗 in Spark NLP 🚀
